# Lab | Langchain Evaluation

## Intro

Pick different sets of data and re-run this notebook. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications.

What did you learn? - Let's discuss that in class

## LangChain: Evaluation

### Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [2]:

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

### Example 1

#### Create our QandA application

In [3]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [4]:
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [5]:
!pip install langchain-community  # Install the langchain-community package using pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00


In [6]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import LLMChain


In [14]:
file = '/content/BooksDatasetClean.csv'
loader = CSVLoader(file_path=file)
data = loader.load()
# Take a subset of the data, e.g., first 100 rows
data = data[:100]

In [18]:
# Print the first 3 documents
for doc in data[:3]:
    print(doc.page_content)
    print('-' * 80)

Title: Goat Brothers
Authors: By Colton, Larry
Description: 
Category: History , General
Publisher: Doubleday
Price Starting With ($): 8.79
Publish Date (Month): January
Publish Date (Year): 1993
--------------------------------------------------------------------------------
Title: The Missing Person
Authors: By Grumbach, Doris
Description: 
Category: Fiction , General
Publisher: Putnam Pub Group
Price Starting With ($): 4.99
Publish Date (Month): March
Publish Date (Year): 1981
--------------------------------------------------------------------------------
Title: Don't Eat Your Heart Out Cookbook
Authors: By Piscatella, Joseph C.
Description: 
Category: Cooking , Reference
Publisher: Workman Pub Co
Price Starting With ($): 4.99
Publish Date (Month): September
Publish Date (Year): 1983
--------------------------------------------------------------------------------


In [ ]:
# !pip install --upgrade --force-reinstall sentence-transformers

In [9]:
!pip install docarray


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 28.3 MB/s eta 0:00:00


In [19]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cuda'})
).from_loaders([loader])

In [22]:
llm = ChatOpenAI(temperature = 0.0,openai_api_key=OPENAI_API_KEY)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

#### Coming up with test datapoints

In [17]:
data[9]

Document(metadata={'source': '/content/BooksDatasetClean.csv', 'row': 9}, page_content='Title: Christmas Cookies\nAuthors: By Eakin, Katherine M. and Deaman, Joane (EDT)\nDescription: \nCategory: Cooking , General\nPublisher: Oxmoor House\nPrice Starting With ($): 12.98\nPublish Date (Month): June\nPublish Date (Year): 1986')

In [23]:
data[11]

Document(metadata={'source': '/content/BooksDatasetClean.csv', 'row': 11}, page_content="Title: Germs : Biological Weapons and America's Secret War\nAuthors: By Miller, Judith, Engelberg, Stephen, and Broad, William J.\nDescription: Deadly germs sprayed in shopping malls, bomb-lets spewing anthrax spores over battlefields, tiny vials of plague scattered in Times Square -- these are the poor man's hydrogen bombs, hideous weapons of mass destruction that can be made in a simple laboratory. In this groundbreaking work of investigative journalism, Judith Miller, Stephen Engelberg, and William Broad of The New York Times uncover the truth about biological weapons and show why bio-warfare and bio-terrorism are fast becoming our worst national nightmare.  Among the startling revelations in Germs:   How the CIA secretly built and tested a model of a Soviet-designed germ bomb, alarming some officials who felt the work pushed to the limits of what is permitted by the global treaty banning germ a

#### Hard-coded examples

In [24]:
from langchain.prompts import PromptTemplate

In [26]:
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI  # or HuggingFaceHub, etc.

# Step 1: Book-related examples
examples = [
    {
        "query": "Who is the author of The Great Gatsby?",
        "answer": "F. Scott Fitzgerald"
    },
    {
        "query": "What genre is 1984?",
        "answer": "Dystopian fiction"
    }
]

# Step 2: Define a book-specific prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template=(
        "You are a helpful assistant for a book database. "
        "Answer questions based on known information.\n\n"
        "Examples:\n"
        "1. Query: Who is the author of The Great Gatsby?\n"
        "   Answer: F. Scott Fitzgerald\n"
        "2. Query: What genre is 1984?\n"
        "   Answer: Dystopian fiction\n"
        "Query: {query}\n"
        "Answer:"
    )
)

# Step 3: Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the book-related query")

# Step 4: Output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Step 5: Initialize the LLM
llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)  # Or use another model

# Step 6: Create the chain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=AnswerOutputParser()
)

# Step 7: Run the chain with a book-related query
query = "What is The Catcher in the Rye about?"
result = llm_chain.run({"query": query})

# Step 8: Show the result
print(result)


<ipython-input-26-57ad58bcb17e>:49: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
<ipython-input-26-57ad58bcb17e>:57: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm_chain.run({"query": query})


answer='The Catcher in the Rye is a novel by J.D. Salinger that follows the story of Holden Caulfield, a teenager who has been expelled from prep school and wanders around New York City. The novel explores themes of teenage angst, alienation, and the search for authenticity.'


#### LLM-Generated examples

In [27]:
from langchain.evaluation.qa import QAGenerateChain

In [29]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(openai_api_key=OPENAI_API_KEY))

In [30]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

In [31]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/usr/local/lib/python3.11/dist-packages/langchain/chains/llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [32]:
new_examples[0]

{'qa_pairs': {'query': 'When was the book "Goat Brothers" published?',
  'answer': '"Goat Brothers" was published in January 1993.'}}

In [33]:
data[0]

Document(metadata={'source': '/content/BooksDatasetClean.csv', 'row': 0}, page_content='Title: Goat Brothers\nAuthors: By Colton, Larry\nDescription: \nCategory: History , General\nPublisher: Doubleday\nPrice Starting With ($): 8.79\nPublish Date (Month): January\nPublish Date (Year): 1993')

In [34]:
d_flattened = [data['qa_pairs'] for data in new_examples]
d_flattened

[{'query': 'When was the book "Goat Brothers" published?',
  'answer': '"Goat Brothers" was published in January 1993.'},
 {'query': 'What is the title of the book and who is the author?',
  'answer': 'The title of the book is "The Missing Person" and the author is Doris Grumbach.'},
 {'query': 'When was the book "Don\'t Eat Your Heart Out Cookbook" published?',
  'answer': 'The book was published in September 1983.'},
 {'query': 'What is the title of the handbook mentioned in the document?',
  'answer': 'The title of the handbook is "When Your Corporate Umbrella Begins to Leak: A Handbook for White Collar Re-Employment."'},
 {'query': 'When was the book "Amy Spangler\'s Breastfeeding: A Parent\'s Guide" published?',
  'answer': 'The book was published in February 1997.'}]

#### Combine examples

In [35]:
# examples += new_example
examples += d_flattened

In [36]:
examples[0]

{'query': 'Who is the author of The Great Gatsby?',
 'answer': 'F. Scott Fitzgerald'}

In [37]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Who is the author of The Great Gatsby?',
 'result': 'The author of "The Great Gatsby" is F. Scott Fitzgerald.'}

### Manual Evaluation - Fun part

In [38]:
import langchain
langchain.debug = True

In [39]:
qa.invoke(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Who is the author of The Great Gatsby?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Who is the author of The Great Gatsby?",
  "context": "Title: The Great Gatsby\nAuthors: By Fitzgerald, F. Scott\nDescription: \nCategory: Fiction , Classics\nPublisher: Wordsworth Editions\nPrice Starting With ($): 4.99\nPublish Date (Month): January\nPublish Date (Year): 1993<<<<>>>>>Title: The Great Gatsby (Masterwork Studies Series)\nAuthors: By Lehan, Richard Daniel\nDescription: A critical introduction to the classic Fitzgerald novel, covering such elements as characterization, plot, symbolism, and setting\nCategory: Literary Criticism , American , General\nPublisher: Twayne Publishers\nPrice Starting With ($): 5.29\nPublish Date (Month

{'query': 'Who is the author of The Great Gatsby?',
 'result': 'The author of "The Great Gatsby" is F. Scott Fitzgerald.'}

In [40]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [41]:
examples += d_flattened

In [42]:
examples

[{'query': 'Who is the author of The Great Gatsby?',
  'answer': 'F. Scott Fitzgerald'},
 {'query': 'What genre is 1984?', 'answer': 'Dystopian fiction'},
 {'query': 'When was the book "Goat Brothers" published?',
  'answer': '"Goat Brothers" was published in January 1993.'},
 {'query': 'What is the title of the book and who is the author?',
  'answer': 'The title of the book is "The Missing Person" and the author is Doris Grumbach.'},
 {'query': 'When was the book "Don\'t Eat Your Heart Out Cookbook" published?',
  'answer': 'The book was published in September 1983.'},
 {'query': 'What is the title of the handbook mentioned in the document?',
  'answer': 'The title of the handbook is "When Your Corporate Umbrella Begins to Leak: A Handbook for White Collar Re-Employment."'},
 {'query': 'When was the book "Amy Spangler\'s Breastfeeding: A Parent\'s Guide" published?',
  'answer': 'The book was published in February 1997.'},
 {'query': 'When was the book "Goat Brothers" published?',
  

In [43]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [44]:
predictions

[{'query': 'Who is the author of The Great Gatsby?',
  'answer': 'F. Scott Fitzgerald',
  'result': 'The author of "The Great Gatsby" is F. Scott Fitzgerald.'},
 {'query': 'What genre is 1984?',
  'answer': 'Dystopian fiction',
  'result': '"1984" by George Orwell is a dystopian novel.'},
 {'query': 'When was the book "Goat Brothers" published?',
  'answer': '"Goat Brothers" was published in January 1993.',
  'result': 'The book "Goat Brothers" was published in January 1993.'},
 {'query': 'What is the title of the book and who is the author?',
  'answer': 'The title of the book is "The Missing Person" and the author is Doris Grumbach.',
  'result': 'The title of the book is "I May Be Wrong but I Doubt It" and the authors are Barkley, Charles and Wilbon, Michael.'},
 {'query': 'When was the book "Don\'t Eat Your Heart Out Cookbook" published?',
  'answer': 'The book was published in September 1983.',
  'result': 'The book "Don\'t Eat Your Heart Out Cookbook" was published in September 1

In [45]:
from langchain.evaluation.qa import QAEvalChain

In [47]:
llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)
eval_chain = QAEvalChain.from_llm(llm)

In [48]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [49]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'}]

In [50]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Who is the author of The Great Gatsby?
Real Answer: F. Scott Fitzgerald
Predicted Answer: The author of "The Great Gatsby" is F. Scott Fitzgerald.

Example 1:
Question: What genre is 1984?
Real Answer: Dystopian fiction
Predicted Answer: "1984" by George Orwell is a dystopian novel.

Example 2:
Question: When was the book "Goat Brothers" published?
Real Answer: "Goat Brothers" was published in January 1993.
Predicted Answer: The book "Goat Brothers" was published in January 1993.

Example 3:
Question: What is the title of the book and who is the author?
Real Answer: The title of the book is "The Missing Person" and the author is Doris Grumbach.
Predicted Answer: The title of the book is "I May Be Wrong but I Doubt It" and the authors are Barkley, Charles and Wilbon, Michael.

Example 4:
Question: When was the book "Don't Eat Your Heart Out Cookbook" published?
Real Answer: The book was published in September 1983.
Predicted Answer: The book "Don't Eat Your Heart Ou

### Example 2
One can also easily evaluate your QA chains with the metrics offered in ragas

In [56]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/index.html

Looking in indexes: https://download.pytorch.org/index.html


In [60]:
from langchain_huggingface import HuggingFaceEmbeddings
loader = TextLoader("nyc_text.txt")
# Try initializing embeddings without specifying device
index = VectorstoreIndexCreator(embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")).from_loaders([loader])

llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)

/usr/local/lib/python3.11/dist-packages/langchain/indexes/vectorstore.py:171: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [61]:
# testing it out

question = "How did New York City get its name?"
result = qa_chain.invoke({"query": question})
result["result"]

'New York City was originally named New Amsterdam by Dutch colonists in 1626. When the city came under British control in 1664, it was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.'

In [62]:
result

{'query': 'How did New York City get its name?',
 'result': 'New York City was originally named New Amsterdam by Dutch colonists in 1626. When the city came under British control in 1664, it was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.',
 'source_documents': [Document(id='d04cde6f-229e-4ba9-b1bc-88adddb65e9f', metadata={'source': 'nyc_text.txt'}, page_content='The city and its metropolitan area constitute the premier gateway for legal immigration to the United States. As many as 800 languages are spoken in New York, making it the most linguistically diverse city in the world. New York City is home to more than 3.2 million residents born outside the U.S., the largest foreign-born population of any city in the world as of 2016.New York City traces its origins to a trading post founded on the southern tip of Manhattan Island by Dutch colonists in approximately 16

Now in order to evaluate the qa system we generated a few relevant questions. We've generated a few question for you but feel free to add any you want.

In [63]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,190",
    "Brooklyn",
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [64]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

#### Introducing RagasEvaluatorChain

`RagasEvaluatorChain` creates a wrapper around the metrics ragas provides (documented [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)), making it easier to run these evaluation with langchain and langsmith.

The evaluator chain has the following APIs

- `__call__()`: call the `RagasEvaluatorChain` directly on the result of a QA chain.
- `evaluate()`: evaluate on a list of examples (with the input queries) and predictions (outputs from the QA chain).
- `evaluate_run()`: method implemented that is called by langsmith evaluators to evaluate langsmith datasets.

lets see each of them in action to learn more.

In [65]:
result = qa_chain.invoke({"query": eval_questions[1]})
result["result"]

'Manhattan (New York County) has the highest population density of any borough in New York City.'

In [66]:
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        result_updated[new_key] = result[old_key]


In [67]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': [Document(id='9f081a62-a65c-4fa0-b157-2c866bb2e052', metadata={'source': 'nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's populatio

In [69]:
!pip install --no-cache-dir recordclass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 177.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for recordclass: filename=recordclass-0.23.1-cp311-cp311-linux_x86_64.whl size=453249 sha256=88b2fc4dd15ac14f0d486b425c13717cab1af94c2b63f91641135276f49f0b58
  Stored in directory: /tmp/pip-ephem-wheel-cache-2f23074l/wheels/23/43/c9/edc2de309801c14a6a7ecb6a38eb3844802c5e895ac3094a01
Successfully built recordclass


In [70]:
!pip install ragas==0.1.9

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [76]:
!pip install ragas==0.1.9

1. `__call__()`

Directly run the evaluation chain with the results from the QA chain. Do note that metrics like context_relevancy and faithfulness require the `source_documents` to be present.

In [85]:
!pip install ragas --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: ragas
    Found existing installation: ragas 0.1.9
    Uninstalling ragas-0.1.9:
      Successfully uninstalled ragas-0.1.9


In [86]:
from ragas.integrations.langchain import RagasEvaluatorChain
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)
from langchain_openai import ChatOpenAI

# ... (rest of your code) ...

faithfulness_chain = RagasEvaluatorChain.from_llm(
    llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY), metric=faithfulness
)
answer_rel_chain = RagasEvaluatorChain.from_llm(
    llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY), metric=answer_relevancy
)
context_rel_chain = RagasEvaluatorChain.from_llm(
    llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY), metric=context_relevancy
)
context_recall_chain = RagasEvaluatorChain.from_llm(
    llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY), metric=context_recall
)


ImportError: cannot import name 'RagasEvaluatorChain' from 'ragas.integrations.langchain' (/usr/local/lib/python3.11/dist-packages/ragas/integrations/langchain.py)

In [ ]:
# Recheck the result that we are going to validate.
result

**Faithfulness**

In [ ]:
eval_result = faithfulness_chain(result_updated)
eval_result["faithfulness_score"]

High faithfulness_score means that there are exact consistency between the source documents and the answer.

You can check lower faithfulness scores by changing the result (answer from LLM) or source_documents to something else.

In [ ]:
fake_result = result.copy()
fake_result["result"] = "we are the champions"
eval_result = faithfulness_chain(fake_result)
eval_result["faithfulness_score"]

**Context Relevancy**

In [ ]:
eval_result = context_recall_chain(result)
eval_result["context_recall_score"]

High context_recall_score means that the ground truth is present in the source documents.

You can check lower context recall scores by changing the source_documents to something else.

In [ ]:
from langchain.schema import Document
fake_result = result.copy()
fake_result["source_documents"] = [Document(page_content="I love christmas")]
eval_result = context_recall_chain(fake_result)
eval_result["context_recall_score"]

2. `evaluate()`

Evaluate a list of inputs/queries and the outputs/predictions from the QA chain.

In [ ]:
# run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

# evaluate
print("evaluating...")
r = faithfulness_chain.evaluate(examples, predictions)
r

In [ ]:
# evaluate context recall
print("evaluating...")
r = context_recall_chain.evaluate(examples, predictions)
r